

**Project Scenario**: You are a Data Engineer working for an online clothing brand that sells a wide range of fashion Brands.

The supply chain team has been tasked with building a dashboard to **Analyze Orders history**. Your dashboard will be used to inform purchasing behaviour and ensure that the company has enough inventory to meet demand for the upcoming holiday season.

Throughout this real-world business scenario, you will learn how to create and ingest data into a delta table. Then use Databricks notebooks (using Python and SQL) to process/transform the data and produce the Supply chain dashboard. At the end you'll leverage Delta Lake's built-in functionalities such as merge operations and time travel to create a scalable data pipeline.

# TASK 1 - Upload project JSON files to Databricks file system

In [ ]:
# First Check that the parameter "DBFS File Browser" is Enabled. Navigate to "Settings > Admin > Workspace settings"  to check

### a. Upload ORDERS json files in Databricks File System

In [ ]:
## Load Data Using the UI to this path dbfs:/FileStore/SupplyChain/ORDERS_RAW/
## Only upload these files: ORDERS_RAW_PART_001.json, ORDERS_RAW_PART_002.json, ORDERS_RAW_PART_003.json, ORDERS_RAW_PART_004.json

### b. Check loaded files

In [ ]:
# Use Databricks Utilities (dbutils). Documentation : https://docs.databricks.com/dev-tools/databricks-utils.html#ls-command-dbutilsfsls 
dbutils.fs.ls("dbfs:/FileStore/SupplyChain/ORDERS_RAW")


Out[5]: [FileInfo(path='dbfs:/FileStore/SupplyChain/ORDERS_RAW/ORDERS_RAW_PART_001.json', name='ORDERS_RAW_PART_001.json', size=260483, modificationTime=1719451817000),
 FileInfo(path='dbfs:/FileStore/SupplyChain/ORDERS_RAW/ORDERS_RAW_PART_002.json', name='ORDERS_RAW_PART_002.json', size=260437, modificationTime=1719451817000),
 FileInfo(path='dbfs:/FileStore/SupplyChain/ORDERS_RAW/ORDERS_RAW_PART_003.json', name='ORDERS_RAW_PART_003.json', size=260640, modificationTime=1719451818000),
 FileInfo(path='dbfs:/FileStore/SupplyChain/ORDERS_RAW/ORDERS_RAW_PART_004.json', name='ORDERS_RAW_PART_004.json', size=4928, modificationTime=1719451818000),
 FileInfo(path='dbfs:/FileStore/SupplyChain/ORDERS_RAW/UPDATE_ORDERS_RAW.json', name='UPDATE_ORDERS_RAW.json', size=2628, modificationTime=1719793248000)]

# TASK 2 - Create Delta Table : ORDERS_RAW

### a. Read multiline json files using spark dataframe:

In [ ]:
# Read multiple line json files using spark dataframeAPI


orders_raw_df = spark.read.option("multiline","true").json("dbfs:/FileStore/SupplyChain/ORDERS_RAW")

## Show the datafarme
orders_raw_df.show(n=5, truncate=False) 

## click on orders_raw_df to Check the schema

+---------------+-------------------+-----+-----------+-------------+----------+--------+------------+-----------------+-----------------------+--------+---------------+--------+------------+----------+
|BRAND          |CATEGORY           |COLOR|CUSTOMER_ID|ORDER_COUNTRY|ORDER_DATE|ORDER_ID|ORDER_STATUS|PAYMENT_METHOD   |PRODUCT_NAME           |QUANTITY|SHIPPING_METHOD|SIZE    |SUB-CATEGORY|UNIT_PRICE|
+---------------+-------------------+-----+-----------+-------------+----------+--------+------------+-----------------+-----------------------+--------+---------------+--------+------------+----------+
|Gap            |Men's Clothing     |Navy |2348       |Germany      |2023-01-11|ORD-200 |Shipped     |Cash on Delivery |Classic Cotton T-Shirt |6       |Express        |Size L  |Tops        |24.99     |
|Adidas Kids    |Kids Clothing      |Green|2149       |Mexico       |2023-01-11|ORD-1418|Delivered   |Credit/Debit Card|Green Hooded Sweatshirt|3       |Standard       |Size 14 |Tops      

In [ ]:
#Validate loaded files, Count Number of Rows in the DataFrame, the total Should be "1510"
orders_raw_df.count()

Out[8]: 1515

### b. Create Delta Table ORDERS_RAW

Delta Lake is 100% compatible with Apache Spark&trade;, which makes it easy to get started with if you already use Spark for your big data workflows.
Delta Lake features APIs for **SQL**, **Python**, and **Scala**, so that you can use it in whatever language you feel most comfortable in.


   <img src="https://databricks.com/wp-content/uploads/2020/12/simplysaydelta.png" width=400/>

In [ ]:
# First, Create Database SupplyChainDB if it doesn't exist
db = "SupplyChainDB"

spark.sql(f"CREATE DATABASE IF NOT EXISTS {db}")
spark.sql(f"USE {db}")

Out[9]: DataFrame[]

In [ ]:
## Create DeltaTable ORDERS_RAW in the metastore using DataFrame's schema and write data to it
## Documentation : https://docs.delta.io/latest/quick-start.html#create-a-table
orders_raw_df.write.mode("overwrite").format("delta").option("overwriteSchema","true").saveAsTable("ORDERS_RAW")

### c. Show Created Delta Table:

In [ ]:
%sql
-- Switch to SQL Cell using %SQL
SHOW tables
 
 -- Alternativerly you can use Python: display(spark.sql(f"SHOW TABLES"))

database,tableName,isTemporary
supplychaindb,orders_raw,false


**d. Validate data loaded successfully to Delta Table ORDERS_RAW**:

In [ ]:
%sql

select count(*) from orders_raw

count(1)
1515


**e. Decsribe Detail of the Delta Table**:

In [ ]:
%sql

describe DETAIL ORDERS_RAW

-- Returns the basic metadata information of a delta table.

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,e0843de1-781a-4b8e-a5ca-d430920cb861,spark_catalog.supplychaindb.orders_raw,null,dbfs:/user/hive/warehouse/supplychaindb.db/orders_raw,2024-06-28T02:08:07.125+0000,2024-07-02T16:38:19.000+0000,List(),5,62631,Map(),1,2,"List(appendOnly, invariants)",Map()


#Activity 1 : Create INVENTORY Delta table

### a. Upload INVENTORY.json file in DBFS

In [ ]:
## Load the file using the UI to this path dbfs:/FileStore/SupplyChain/INVENTORY/

###b. Read the File using spark dataframe

In [ ]:
inventory_df = spark.read.option("multiline","true").json("dbfs:/FileStore/SupplyChain/INVENTORY")

## Show the datafarme
inventory_df.show(n=5, truncate=False)

+-------+----------+--------------------------+----------+-----+
|BRAND  |COLOR     |PRODUCT_NAME              |SIZE      |STOCK|
+-------+----------+--------------------------+----------+-----+
|J.Crew |Green     |Green Cargo Pants         |Size 32x32|58   |
|Theory |Grey      |Grey Turtleneck Sweater   |Size S    |42   |
|Ray-Ban|Gold/Brown|Classic Aviator Sunglasses|One Size  |53   |
|ASOS   |Black     |Men's Faux Leather Jacket |Size M    |40   |
|Levi's |Light Blue|Distressed Denim Shorts   |Size M    |46   |
+-------+----------+--------------------------+----------+-----+
only showing top 5 rows



### c. Create Delta Table INVENTORY

In [ ]:
# First, Create Database SupplyChainDB
db = "SupplyChainDB"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db}")
spark.sql(f"USE {db}")

Out[17]: DataFrame[]

In [ ]:
## Create INVENTORY Delta Table 
inventory_df.write.mode("overwrite").format("delta").option("overwriteSchema","true").saveAsTable("INVENTORY")

### d. Show Created Delta Tables:

In [ ]:
%sql
-- Switch to SQL Cell using %sql
SHOW TABLES

database,tableName,isTemporary
supplychaindb,inventory,false
supplychaindb,orders_raw,false


# TASK 3 - Transform data in delta table

<a href="https://www.databricks.com/glossary/medallion-architecture" target="_blank">Medallion Architecture</a>   
</br>
<img src="https://databricks.com/wp-content/uploads/2020/09/delta-lake-medallion-model-scaled.jpg" width=900/>

During this Task you will : 
* 1- Read delta Table using Spark Dataframe
* 2- Convert Data Type String --> Date
* 3- Drop Rows with Null Values
* 4- Add a Computed Column "TOTAL_ORDER"
* 5- Create new deltatable Orders_Gold

### a. Read ORDERS_RAW delta table using spark Dataframe

In [ ]:
#read Delta Table using spark dataframe
#read orders_raw table into silver layer i.e orders_silver_df
ORDERS_Gold_df=  spark.read.table("supplychaindb.orders_raw")

ORDERS_Gold_df.show(n=5,truncate=False)
# Click on ORDERS_DF to See the Schema of the Table. 

+------------+----------------+--------------+-----------+-------------+----------+--------+------------+-----------------+---------------------+--------+---------------+-------+------------+----------+
|BRAND       |CATEGORY        |COLOR         |CUSTOMER_ID|ORDER_COUNTRY|ORDER_DATE|ORDER_ID|ORDER_STATUS|PAYMENT_METHOD   |PRODUCT_NAME         |QUANTITY|SHIPPING_METHOD|SIZE   |SUB-CATEGORY|UNIT_PRICE|
+------------+----------------+--------------+-----------+-------------+----------+--------+------------+-----------------+---------------------+--------+---------------+-------+------------+----------+
|H&M Kids    |Kids Clothing   |Pink and Green|2066       |Hong Kong    |2022-01-21|ORD-1281|Processing  |Credit/Debit Card|Pink Floral Dress    |3       |Standard       |Size 6 |Dresses     |24.99     |
|H&M         |Women's Clothing|Cream         |2254       |Spain        |2022-01-23|ORD-541 |Delivered   |Credit/Debit Card|Women's Faux Fur Coat|4       |Standard       |Size M |Outerwear 

### b. Update ORDER_DATE Column's Data Type

In [ ]:
# Use withColumn method & to_date()
# withColumn Documentation : https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.DataFrame.withColumn.html
# TO_DATE() Documentation : https://docs.databricks.com/sql/language-manual/functions/to_date.html

from pyspark.sql.functions import *


ORDERS_Gold_df =  ORDERS_Gold_df.withColumn("ORDER_DATE",to_date(col("ORDER_DATE"),"yyyy-MM-dd"))

### c. Drop Rows with Null Values

In [ ]:
# Count Nulls for each column
from pyspark.sql.functions import *

display(ORDERS_Gold_df.select([count(when(col(c).isNull(),c)).alias(c) for c in ORDERS_Gold_df.columns]))

BRAND,CATEGORY,COLOR,CUSTOMER_ID,ORDER_COUNTRY,ORDER_DATE,ORDER_ID,ORDER_STATUS,PAYMENT_METHOD,PRODUCT_NAME,QUANTITY,SHIPPING_METHOD,SIZE,SUB-CATEGORY,UNIT_PRICE
0,0,0,10,10,0,10,0,0,10,10,0,0,0,0


In [ ]:
#  Remove Nulls using dropna() method which removes all rows with Null Values 

ORDERS_Gold_df = ORDERS_Gold_df.dropna()

ORDERS_Gold_df.count()

Out[26]: 1505

### d. Add new Column TOTAL_ORDER

In [ ]:
#Use withColumn function
#Documentation : https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.DataFrame.withColumn.html


ORDERS_Gold_df= ORDERS_Gold_df.withColumn("TOTAL_ORDER",col("QUANTITY")*col("UNIT_PRICE"))

# Display ORDERS_Gold_df to validate the creation of the New Column TOTAL_ORDER
display(ORDERS_Gold_df)

BRAND,CATEGORY,COLOR,CUSTOMER_ID,ORDER_COUNTRY,ORDER_DATE,ORDER_ID,ORDER_STATUS,PAYMENT_METHOD,PRODUCT_NAME,QUANTITY,SHIPPING_METHOD,SIZE,SUB-CATEGORY,UNIT_PRICE,TOTAL_ORDER
H&M Kids,Kids Clothing,Pink and Green,2066,Hong Kong,2022-01-21,ORD-1281,Processing,Credit/Debit Card,Pink Floral Dress,3,Standard,Size 6,Dresses,24.99,74.97
H&M,Women's Clothing,Cream,2254,Spain,2022-01-23,ORD-541,Delivered,Credit/Debit Card,Women's Faux Fur Coat,4,Standard,Size M,Outerwear,129.99,519.96
Canada Goose,Men's Clothing,Dark Green,2033,Hong Kong,2022-01-23,ORD-1388,Shipped,Credit/Debit Card,Men's Parka,2,Standard,Size XL,Jackets,999.99,1999.98
Zara Kids,Kids Clothing,Grey,2144,Switzerland,2022-01-24,ORD-1158,Delivered,Credit/Debit Card,Grey Hoodie,1,Standard,Size L,Sweatshirts,39.99,39.99
Canada Goose,Men's Clothing,Green,2001,Germany,2022-01-24,ORD-1351,Shipped,Credit/Debit Card,Green Parka,2,Standard,Size L,Outerwear,699.99,1399.98
Mango,Women's Clothing,Pink/White,2360,South Africa,2022-01-25,ORD-665,Delivered,Credit/Debit Card,Floral Midi Dress,2,Standard,Size 6,Dresses,148.0,296.0
H&M Kids,Kids Clothing,Orange,2128,Canada,2022-01-26,ORD-428,Cancelled,PayPal,Orange Cargo Shorts,5,Standard,Size 8,Shorts,22.99,114.94999999999999
Gap,Men's Clothing,Navy,2406,Egypt,2022-01-26,ORD-760,Shipped,Cash on Delivery,Classic Cotton T-Shirt,7,Express,Size L,Tops,24.99,174.92999999999998
Zara,Men's Clothing,Green,2365,Mexico,2022-01-26,ORD-384,Cancelled,PayPal,Green Utility Jacket,7,Standard,Size XL,Jackets,129.99,909.9300000000001
H&M Kids,Kids Clothing,Gray,2027,Italy,2022-01-27,ORD-567,Shipped,Credit/Debit Card,Gray Sweatshirt,5,Standard,Size 6,Tops,19.99,99.94999999999999


### e. Create Delta Table ORDERS_GOLD

In [ ]:
# Make sure you are using SupplyChainDB
spark.sql(f"USE SupplyChainDB")

## Create DeltaTable Orders_GOLD: 

ORDERS_Gold_df.write.mode("overwrite").format("delta").option("overwriteSchema","true").saveAsTable("ORDERS_GOLD")


## Validate that the table was created successfully
display(spark.sql(f"SHOW TABLES"))

database,tableName,isTemporary
supplychaindb,inventory,false
supplychaindb,orders_gold,false
supplychaindb,orders_raw,false


-- Read more about different write options and parameters here https://docs.delta.io/latest/delta-batch.html#write-to-a-table 

* **Append** to automatically add new data to an existing Delta table, 
* **Overwrite** To automatically replace all the data in a table:

# TASK 5 - Query Orders Delta table using SQL

### Get Familiar with Orders_Gold dataset

In [ ]:
%sql
-- Get top 30 rows Get Familiar with the Data
select * from supplychaindb.orders_gold limit 10;


BRAND,CATEGORY,COLOR,CUSTOMER_ID,ORDER_COUNTRY,ORDER_DATE,ORDER_ID,ORDER_STATUS,PAYMENT_METHOD,PRODUCT_NAME,QUANTITY,SHIPPING_METHOD,SIZE,SUB-CATEGORY,UNIT_PRICE,TOTAL_ORDER
H&M Kids,Kids Clothing,Pink and Green,2066,Hong Kong,2022-01-21,ORD-1281,Processing,Credit/Debit Card,Pink Floral Dress,3,Standard,Size 6,Dresses,24.99,74.97
H&M,Women's Clothing,Cream,2254,Spain,2022-01-23,ORD-541,Delivered,Credit/Debit Card,Women's Faux Fur Coat,4,Standard,Size M,Outerwear,129.99,519.96
Canada Goose,Men's Clothing,Dark Green,2033,Hong Kong,2022-01-23,ORD-1388,Shipped,Credit/Debit Card,Men's Parka,2,Standard,Size XL,Jackets,999.99,1999.98
Zara Kids,Kids Clothing,Grey,2144,Switzerland,2022-01-24,ORD-1158,Delivered,Credit/Debit Card,Grey Hoodie,1,Standard,Size L,Sweatshirts,39.99,39.99
Canada Goose,Men's Clothing,Green,2001,Germany,2022-01-24,ORD-1351,Shipped,Credit/Debit Card,Green Parka,2,Standard,Size L,Outerwear,699.99,1399.98
Mango,Women's Clothing,Pink/White,2360,South Africa,2022-01-25,ORD-665,Delivered,Credit/Debit Card,Floral Midi Dress,2,Standard,Size 6,Dresses,148.0,296.0
H&M Kids,Kids Clothing,Orange,2128,Canada,2022-01-26,ORD-428,Cancelled,PayPal,Orange Cargo Shorts,5,Standard,Size 8,Shorts,22.99,114.94999999999999
Gap,Men's Clothing,Navy,2406,Egypt,2022-01-26,ORD-760,Shipped,Cash on Delivery,Classic Cotton T-Shirt,7,Express,Size L,Tops,24.99,174.92999999999998
Zara,Men's Clothing,Green,2365,Mexico,2022-01-26,ORD-384,Cancelled,PayPal,Green Utility Jacket,7,Standard,Size XL,Jackets,129.99,909.9300000000001
H&M Kids,Kids Clothing,Gray,2027,Italy,2022-01-27,ORD-567,Shipped,Credit/Debit Card,Gray Sweatshirt,5,Standard,Size 6,Tops,19.99,99.94999999999999


### KPI-1: Quantity Sold by Country

In [ ]:
%sql
-- Division = CATEGORY 
-- Dont forget to Filter out Cancelled Orders
-- Avnish -- Put this data into Gold layer
select order_country, sum(quantity) as total_demand from orders_gold where order_status != 'Cancelled' group by order_country


order_country,total_demand
Germany,278
France,243
Greece,252
India,229
United States,215
China,211
Italy,181
Norway,244
Spain,213
Denmark,185


Databricks visualization. Run in Databricks to view.

### KPI-2: Sales by Division ($)

In [ ]:
%sql
-- Dont forget to Filter out Cancelled Orders
select category, sum(total_order) as revenue from orders_gold where order_status != 'Cancelled' group by category order by revenue desc


category,revenue
Men's Clothing,403902.19000000006
Women's Clothing,276763.86000000004
Accessories,99434.47000000002
Kids Clothing,85777.13999999997
Men's Shoes,50567.42
Men's Accessories,31758.91
Women's Accessories,21388.93
Women's Shoes,10349.130000000001
Unisex Accessories,6899.540000000001


Databricks visualization. Run in Databricks to view.

### KPI-3: Top-5 Popular Brands

In [ ]:
%sql
-- Limit Result to 5 and Order Results by Sold Quanity
select brand, sum(quantity) as total_sold_items from supplychaindb.orders_gold group by brand order by total_sold_items desc limit 5


brand,total_sold_items
Mango,682
Coach,436
Zara,418
Nike Kids,381
H&M Kids,369


Databricks visualization. Run in Databricks to view.

# TASK 6 - Create Dashboard

In [ ]:
# Use Databricks UI
# 1- Turn results of Previous Queries into visualisations
# 2- Create Dashboard and add Visualisations

# Activity 2 : Add Monthly Sales Trend to your Dashboard

### KPI-4: Monthly Sales Trend (In QTY)

** Instructions :**  
  # 1- Query Delta Table: Orders_Gold to extract Monthly Sales (in Quantity, across all brands and all regions) 
  # 2- Turn the result (Table) into a visualisation (line chart) to Show the Trend for the last 18 months.
  # 3- Add your visualization to the Supply Chain Dashboard.

In [ ]:
%sql
-- Use DATE_TRUNC()  
select date_trunc('month',order_date) as month, sum(quantity) as demand from supplychaindb.orders_gold 
where ORDER_STATUS != 'Cancelled'
group by 1 order by 1 asc 




month,demand
2022-01-01T00:00:00.000+0000,71
2022-02-01T00:00:00.000+0000,156
2022-03-01T00:00:00.000+0000,192
2022-04-01T00:00:00.000+0000,167
2022-05-01T00:00:00.000+0000,200
2022-06-01T00:00:00.000+0000,198
2022-07-01T00:00:00.000+0000,433
2022-08-01T00:00:00.000+0000,332
2022-09-01T00:00:00.000+0000,479
2022-10-01T00:00:00.000+0000,456


Databricks visualization. Run in Databricks to view.

# TASK 7 - Update Data in Orders table using Merge

<img src="https://databricks.com/wp-content/uploads/2020/09/delta-lake-medallion-model-scaled.jpg" width=1012/>

### a. Upload json files into DBFS

Use UI to upload the file "UPDATE_ORDERS_RAW.json" into DBFS, use the same folder dbfs:/FileStore/SupplyChain/ORDERS_RAW/

### b. Read file using Spark dataframe

In [ ]:
# Read multiple line json file UPDATE_ORDERS_RAW.json
Update_orders_df = spark.read.option("multiline","true").json("dbfs:/FileStore/SupplyChain/ORDERS_RAW/UPDATE_ORDERS_RAW.json")

## Show the datafarme
display(Update_orders_df)

BRAND,CATEGORY,COLOR,CUSTOMER_ID,ORDER_COUNTRY,ORDER_DATE,ORDER_ID,ORDER_STATUS,PAYMENT_METHOD,PRODUCT_NAME,QUANTITY,SHIPPING_METHOD,SIZE,SUB-CATEGORY,UNIT_PRICE
H&M Kids,Kids Clothing,Pink and Green,2066,Hong Kong,2022-01-21,ORD-1281,Delivered,Credit/Debit Card,Pink Floral Dress,4,Standard,Size 6,Dresses,24.99
Mango,Women's Clothing,Black,2023,Saudi Arabia,2022-01-28,ORD-829,Delivered,Credit/Debit Card,Women's Leather Moto Jacket,3,Standard,Size S,Jackets,199.99
Madewell,Women's Clothing,Blue,2041,Saudi Arabia,2022-01-28,ORD-193,Delivered,Cash on Delivery,Blue Denim Jacket,3,Standard,Size M,Jackets,99.99
Barbour,Men's Clothing,Navy,2074,Norway,2022-05-29,ORD-826,Cancelled,Credit/Debit Card,Men's Quilted Jacket,0,Standard,Size L,Jackets,299.99
Gap Kids,Kids Clothing,Red,2393,Saudi Arabia,2022-05-30,ORD-842,Cancelled,Credit/Debit Card,Red Graphic T-shirt,0,Standard,Size 8,Tops,14.99


-->Check the original data **BEFORE MERGE**

In [ ]:
%sql 
select ORDER_ID,ORDER_STATUS,Quantity from Supplychaindb.ORDERS_RAW WHERE ORDER_ID in ("ORD-1281","ORD-829","ORD-193","ORD-826","ORD-842")

ORDER_ID,ORDER_STATUS,Quantity
ORD-1281,Processing,3
ORD-829,Processing,3
ORD-193,Shipped,1
ORD-826,Processing,10
ORD-842,Processing,10
ORD-1281,Delivered,4
ORD-829,Delivered,3
ORD-193,Delivered,3
ORD-826,Cancelled,0
ORD-842,Cancelled,0


### c. Update Orders_RAW delta table using Merge

In [ ]:
%sql
DESCRIBE DETAIL supplychaindb.ORDERS_RAW

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,e0843de1-781a-4b8e-a5ca-d430920cb861,spark_catalog.supplychaindb.orders_raw,null,dbfs:/user/hive/warehouse/supplychaindb.db/orders_raw,2024-06-28T02:08:07.125+0000,2024-07-02T00:18:13.000+0000,List(),5,62631,Map(),1,2,"List(appendOnly, invariants)",Map()


In [ ]:
from delta.tables import *

# programmatically interacting with Delta tables using the class delta.tables.DeltaTable(spark: pyspark.sql.session.SparkSession, jdt: JavaObject)
# delta_orders_raw is a dataframe
delta_orders_raw =  DeltaTable.forPath(spark,'dbfs:/user/hive/warehouse/supplychaindb.db/orders_raw')

In [ ]:
## merge data into delta Table ORDER_RAW
# DOCUMENTATION https://docs.delta.io/latest/delta-update.html#language-python 

delta_orders_raw.alias("ORDERS_RAW").merge(Update_orders_df.alias("UpdateOrders"),
                                           "ORDERS_RAW.ORDER_ID = UpdateOrders.ORDER_ID")\
                                           .whenMatchedUpdateAll()\
                                           .whenNotMatchedInsertAll()\
                                          .execute()
                                          

# must be at least one WHEN clause in a MERGE statement.

--> check the udapted rows **AFTER MERGE**

In [ ]:
%sql 
select ORDER_ID,ORDER_STATUS,Quantity from SUPPLYCHAINDB.ORDERS_RAW WHERE ORDER_ID in ("ORD-1281","ORD-829","ORD-193","ORD-826","ORD-842")



ORDER_ID,ORDER_STATUS,Quantity
ORD-1281,Delivered,4
ORD-1281,Delivered,4
ORD-829,Delivered,3
ORD-829,Delivered,3
ORD-193,Delivered,3
ORD-193,Delivered,3
ORD-826,Cancelled,0
ORD-826,Cancelled,0
ORD-842,Cancelled,0
ORD-842,Cancelled,0


Learn More about Merge Operations check out https://docs.delta.io/latest/delta-update.html#language-python

# TASK 8 - Query previous versions of delta table using **Time Travel**

**This Task shows how to time travel between different versions of a Delta table with Delta Lake. You can time travel by table version or by timestamp. You’ll learn about the benefits of time travel and why it’s an essential feature for production data workloads.**

**Documentation : https://delta.io/blog/2023-02-01-delta-lake-time-travel/** 

<img src="https://delta.io/static/9c42ea9f028932de03257ed75d35a8ba/cf8e5/image1.png" width=1012/>

### a. Describe Detla Table History:

In [ ]:
%sql
-- Check Table History 
describe history supplychaindb.orders_raw
-- Use the UI to see Delta Table History

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
18,2024-07-02T00:19:43.000+0000,6874844908570099,avnishlearnsgcp@gmail.com,MERGE,"Map(predicate -> [""(ORDER_ID#6432 = ORDER_ID#6163)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}], notMatchedBySourcePredicates -> [])",null,List(2136992527233995),0702-001426-nmbhsymn,17,WriteSerializable,false,"Map(numTargetRowsCopied -> 495, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, numTargetBytesAdded -> 23040, numTargetBytesRemoved -> 22856, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 10, executionTimeMs -> 10274, numTargetRowsInserted -> 0, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 4274, numTargetRowsUpdated -> 10, numOutputRows -> 505, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 5, numTargetFilesRemoved -> 2, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 5090)",null,Databricks-Runtime/12.2.x-scala2.12
17,2024-07-02T00:18:13.000+0000,6874844908570099,avnishlearnsgcp@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(2136992527233995),0702-001426-nmbhsymn,16,WriteSerializable,false,"Map(numFiles -> 5, numOutputRows -> 1515, numOutputBytes -> 62631)",null,Databricks-Runtime/12.2.x-scala2.12
16,2024-07-01T16:57:51.000+0000,6874844908570099,avnishlearnsgcp@gmail.com,MERGE,"Map(predicate -> [""(ORDER_ID#23439 = ORDER_ID#23170)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}], notMatchedBySourcePredicates -> [])",null,List(2136992527233995),0701-160550-3l4hvbag,15,WriteSerializable,false,"Map(numTargetRowsCopied -> 495, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, numTargetBytesAdded -> 23040, numTargetBytesRemoved -> 22856, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 10, executionTimeMs -> 4907, numTargetRowsInserted -> 0, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 1812, numTargetRowsUpdated -> 10, numOutputRows -> 505, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 5, numTargetFilesRemoved -> 2, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 2691)",null,Databricks-Runtime/12.2.x-scala2.12
15,2024-07-01T16:57:08.000+0000,6874844908570099,avnishlearnsgcp@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(2136992527233995),0701-160550-3l4hvbag,14,WriteSerializable,false,"Map(numFiles -> 5, numOutputRows -> 1515, numOutputBytes -> 62631)",null,Databricks-Runtime/12.2.x-scala2.12
14,2024-07-01T16:46:09.000+0000,6874844908570099,avnishlearnsgcp@gmail.com,MERGE,"Map(predicate -> [""(ORDER_ID#15490 = ORDER_ID#15221)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}], notMatchedBySourcePredicates -> [])",null,List(2136992527233995),0701-160550-3l4hvbag,13,WriteSerializable,false,"Map(numTargetRowsCopied -> 495, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, numTargetBytesAdded -> 23040, numTargetBytesRemoved -> 22856, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 10, executionTimeMs -> 5634, numTargetRowsInserted -> 0, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 2509, numTargetRowsUpdated -> 10, numOutputRows -> 505, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 5, numTargetFilesRemoved -> 2, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 2733)",null,Databricks-Runtime/12.2.x-scala2.12
13,2024-07-01T16:45:22.000+0000,6874844908570099,

### b. Using SQL:

In [ ]:
%sql 
 select ORDER_ID,ORDER_STATUS,Quantity from SUPPLYCHAINDB.ORDERS_RAW VERSION AS OF 5 WHERE ORDER_ID in ("ORD-1281","ORD-829","ORD-193","ORD-826","ORD-842")

-- CHange Version Number to See different Versions of the delta table

ORDER_ID,ORDER_STATUS,Quantity
ORD-1281,Processing,3
ORD-829,Processing,3
ORD-193,Shipped,1
ORD-826,Processing,10
ORD-842,Processing,10
ORD-1281,Delivered,4
ORD-829,Delivered,3
ORD-193,Delivered,3
ORD-826,Cancelled,0
ORD-842,Cancelled,0


### c. Using Spark dataframe:

In [ ]:
#Time Travel
version_1 = spark.read.format('delta').option('TimeStamp', "2023-05-16").table("SUPPLYCHAINDB.ORDERS_RAW")
display(version_1)

BRAND,CATEGORY,COLOR,CUSTOMER_ID,ORDER_COUNTRY,ORDER_DATE,ORDER_ID,ORDER_STATUS,PAYMENT_METHOD,PRODUCT_NAME,QUANTITY,SHIPPING_METHOD,SIZE,SUB-CATEGORY,UNIT_PRICE
H&M,Women's Clothing,Cream,2254,Spain,2022-01-23,ORD-541,Delivered,Credit/Debit Card,Women's Faux Fur Coat,4,Standard,Size M,Outerwear,129.99
Canada Goose,Men's Clothing,Dark Green,2033,Hong Kong,2022-01-23,ORD-1388,Shipped,Credit/Debit Card,Men's Parka,2,Standard,Size XL,Jackets,999.99
Zara Kids,Kids Clothing,Grey,2144,Switzerland,2022-01-24,ORD-1158,Delivered,Credit/Debit Card,Grey Hoodie,1,Standard,Size L,Sweatshirts,39.99
Canada Goose,Men's Clothing,Green,2001,Germany,2022-01-24,ORD-1351,Shipped,Credit/Debit Card,Green Parka,2,Standard,Size L,Outerwear,699.99
Mango,Women's Clothing,Pink/White,2360,South Africa,2022-01-25,ORD-665,Delivered,Credit/Debit Card,Floral Midi Dress,2,Standard,Size 6,Dresses,148.0
H&M Kids,Kids Clothing,Orange,2128,Canada,2022-01-26,ORD-428,Cancelled,PayPal,Orange Cargo Shorts,5,Standard,Size 8,Shorts,22.99
Gap,Men's Clothing,Navy,2406,Egypt,2022-01-26,ORD-760,Shipped,Cash on Delivery,Classic Cotton T-Shirt,7,Express,Size L,Tops,24.99
Zara,Men's Clothing,Green,2365,Mexico,2022-01-26,ORD-384,Cancelled,PayPal,Green Utility Jacket,7,Standard,Size XL,Jackets,129.99
H&M Kids,Kids Clothing,Gray,2027,Italy,2022-01-27,ORD-567,Shipped,Credit/Debit Card,Gray Sweatshirt,5,Standard,Size 6,Tops,19.99
J.Crew,Men's Clothing,Green,2092,United Kingdom,2022-01-27,ORD-1590,Delivered,Credit/Debit Card,Green Cargo Pants,1,Standard,Size 32x32,Pants,89.99


# END OF THE PROJECT

# CUMULATIVE CHALLENGE

**Your Task :</br> Using the “Inventory” data, your task is to enrich the Supply Chain Dashboard with the list of low-stock and out-of-stock Items.** 

Using Databricks notebook you will : </br>
1-Upload INVENTORY.JSON file to DBFS(1) </br>
2-Read the file using spark dataframe (1)</br>
3-Create Delta Table INVENTORY (1)</br>
4-Write an SQL query to join ORDERS_GOLD and INVENTORY DeltaTables to find the list of Items Low-in Stock or Out-of Stock</br>
5-Turn the result into a Visualisation (Table type) and Add it to your SupplyChain Dashboard</br>



### a. Upload INVENTORY.json file in DBFS

In [ ]:
## Load the file using the UI to this path dbfs:/FileStore/SupplyChain/INVENTORY/

### b. Read the File using spark dataframe

In [ ]:
inventory_df = spark.read.option("multiline","true").json("dbfs:/FileStore/SupplyChain/INVENTORY")

## Show the datafarme
inventory_df.show(n=5, truncate=False)

+-------+----------+--------------------------+----------+-----+
|BRAND  |COLOR     |PRODUCT_NAME              |SIZE      |STOCK|
+-------+----------+--------------------------+----------+-----+
|J.Crew |Green     |Green Cargo Pants         |Size 32x32|58   |
|Theory |Grey      |Grey Turtleneck Sweater   |Size S    |42   |
|Ray-Ban|Gold/Brown|Classic Aviator Sunglasses|One Size  |53   |
|ASOS   |Black     |Men's Faux Leather Jacket |Size M    |40   |
|Levi's |Light Blue|Distressed Denim Shorts   |Size M    |46   |
+-------+----------+--------------------------+----------+-----+
only showing top 5 rows



### c. Create Delta Table INVENTORY <img src="https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png" width=35 height=35/>

In [ ]:
# Use SupplyChainDB Database
db = "SupplyChainDB"
spark.sql(f"USE {db}")


Out[40]: DataFrame[]

In [ ]:
## Create INVENTORY Delta Table 
inventory_df.write.mode("overwrite").format("delta").option("overwriteSchema","true").saveAsTable("INVENTORY")

## Validate that the table was created successfully
display(spark.sql(f"SHOW TABLES"))

database,tableName,isTemporary
supplychaindb,inventory,false
supplychaindb,orders_gold,false
supplychaindb,orders_raw,false


### d. Join ORDERS_GOLD and INVENTORY DeltaTables to find the list of Low Stock or Out-of Stock Items

**Your Goal** is to find the list of Low-Stock or Out-of-Stock Items and Add the result to your SupplyChain Dashboard<br />
**Hints:**
* Group all Orders (from ORDERS_GOLD) based-on BRAND, COLOR, PRODUCT_NAME AND SIZE And Add QTY_SOLD (SUM QUANTITY) 
* Join the result with INVENTORY using an inner join on BRAND, PRODUCT_NAME, COLOR AND SIZE.
* Add calculated column "QTY_LEFT_STOCK" as (STOCK - QTY_SOLD)
* Filter-out Cancelled ORDERS (ORDER_STATUS)
* Keep only Items with QTY_LEFT_STOCK < 20
* Sort the result by "QTY_LEFT_STOCK" in ascending order

In [ ]:
%sql
-- Write Your Query Here : 
select J.* from
(select O.*, I.stock, (I.STOCK - O.qty_sold) as QTY_LEFT_STOCK from 
(select brand, color, product_name, size, sum(quantity) as qty_sold
from supplychaindb.orders_gold
where ORDER_STATUS != 'Cancelled'
group by brand, color, product_name, size) O
join supplychaindb.inventory I
on (O.brand = I.BRAND)
and (O.product_name = I.PRODUCT_NAME)
and (O.color = I.COLOR)
and (O.size = I.SIZE)) J
where J.QTY_LEFT_STOCK < 20
order by J.QTY_LEFT_STOCK asc


brand,color,product_name,size,qty_sold,stock,QTY_LEFT_STOCK
ASOS,Black,Men's Faux Leather Jacket,Size M,37,40,3
Gap,Navy,Classic Cotton T-Shirt,Size L,40,44,4
Theory,Grey,Grey Turtleneck Sweater,Size S,37,42,5
J.Crew,Green,Green Cargo Pants,Size 32x32,52,58,6
Ray-Ban,Gold/Brown,Classic Aviator Sunglasses,One Size,46,53,7
Steve Madden,Black,Lace-Up Combat Boots,Size 8,55,65,10
Levi's,Light Blue,Distressed Denim Shorts,Size M,36,46,10
Coach,Black,Leather Crossbody Bag,One Size,40,53,13
Nike Kids,Gray,Gray Joggers,Size 12,36,50,14
Dr. Martens,Black,Black Leather Chelsea Boots,Size 10,45,61,16


Databricks visualization. Run in Databricks to view.

### e. Turn the result into a Visualisation (Table) and Add it to SupplyChain Dashboard